### Data processing scripts
Includes functions for image padding, data augmentation and undersampling

by Loic Baum

In [ ]:
import cv2
import os
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
%matplotlib inline

### Augmenting Data from Test Dataset
This dataset is quite interesting because all the images have various backgrounds.
The problem is that the dataset only contains 870. 
We will use data augmentation to have more data to train on

In [ ]:
def augment_images_from_dir(directory,output_directory,nb_data_aug):
    aug = iaa.SomeOf(3, [
    iaa.Rotate((-10,10)),
    iaa.MultiplyBrightness((0.70, 1.30)),
    iaa.Crop(percent=(0, 0.10)),
    iaa.TranslateX(percent=(-0.1,0.1)),
    iaa.TranslateY(percent=(-0.1,0.1)),
    iaa.ShearX((-10,10)),
    iaa.ShearY((-10,10)),
    iaa.AdditiveGaussianNoise(scale=(0,10))])
    
    uniq_labels = sorted(os.listdir(directory))
    for idx, label in enumerate(uniq_labels): #going through the labels folders
        print("|",label,"|",end="")
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            image = cv2.imread(filepath)
            for i in range(nb_data_aug):
                img_aug = aug(image=image)
                cv2.imwrite(output_directory+"/"+label+"/aug_"+str(i)+"_"+file,img_aug)

input_test_dir = "./asl-alphabet-test/asl-alphabet-test"
output_dir = "././asl-alphabet-augmented-balanced/"

augment_images_from_dir(input_test_dir,output_dir,40)

### Training set undersampling
The initial training set is very large (87 000 images), but all the images were extracted from a video
so the picture are very similar. In order to have a balanced dataset between images with background
and images without background, we will undersample the initial training set to 1/3 of its size.

In [9]:
def undersample_dataset(input_directory, output_directory):
    cnt=0
    uniq_labels = sorted(os.listdir(input_directory))
    print("Undersampling Folder :",end="")
    for idx, label in enumerate(uniq_labels): #going through the labels folders
        print("|",label,"|",end="")
        for file in os.listdir(input_directory + "/" + label):
            cnt+=1
            if(cnt==3):
                filepath = input_directory + "/" + label + "/" + file
                image = cv2.imread(filepath)
                cv2.imwrite(output_directory+"/"+label+"/_us_"+file ,image)
                cnt=0


input_train_dir = "./asl-alphabet/asl_alphabet_train/asl_alphabet_train"

undersample_dataset(input_train_dir,output_dir)
    

Undersampling Folder :| A || B || C || D || E || F || G || H || I || J || K || L || M || N || O || P || Q || R || S || T || U || V || W || X || Y || Z || del || nothing || space |

### Process image from Surrey Dataset :
Images need to be resized to 64x64, but most of the images have a rectangular shape (ex 70x120)
To avoid distortion, we will add padding to the images to have a square size (ex : 120x120) and then we will resize them to 64x64

In [2]:
import glob

def pad_and_resize_dataset(input_directory, output_directory, img_size):
    folder_list = sorted(os.listdir(input_directory))
    for i,folder in enumerate(folder_list): #going through the 5 folders of the dataset  
        print("Folder :", folder)
        label_list = sorted(os.listdir(input_directory+'/'+folder))
        for j,label in enumerate(label_list): #going through the labels folders
            print("|",label,"|",end="")
            for k,file in enumerate(glob.glob(input_directory+ "/" +folder+'/'+ label +'/'+'color_*.png')):
                image = cv2.imread(file)
                #get height and witdh of read image
                h = image.shape[0]
                w = image.shape[1]
                if h>w:
                    border_size = int((h-w)/2)
                    image = cv2.copyMakeBorder(image,0,0,border_size, border_size,cv2.BORDER_CONSTANT)
                elif w>h:
                    border_size = int((w-h)/2)                    
                    image = cv2.copyMakeBorder(image, border_size, border_size, 0, 0,cv2.BORDER_CONSTANT)
                #images have now a square shape, we can resize them without squeezing them
                image = cv2.resize(image, img_size)
                #write image to new folder
                cv2.imwrite(output_directory+"/"+str(label).capitalize()+"/"+str(i)+'_'+str(k)+'.png', image)
        
pad_and_resize_dataset('../dataset5','./asl-surrey-black-padding',(64,64))          


Folder : A
| a || b || c || d || e || f || g || h || i || k || l || m || n || o || p || q || r || s || t || u || v || w || x || y |Folder : B
| a || b || c || d || e || f || g || h || i || k || l || m || n || o || p || q || r || s || t || u || v || w || x || y |Folder : C
| a || b || c || d || e || f || g || h || i || k || l || m || n || o || p || q || r || s || t || u || v || w || x || y |Folder : D
| a || b || c || d || e || f || g || h || i || k || l || m || n || o || p || q || r || s || t || u || v || w || x || y |Folder : E
| a || b || c || d || e || f || g || h || i || k || l || m || n || o || p || q || r || s || t || u || v || w || x || y |